## N-Body Differentiable PP Solver — Inverse Problem Sampling with HMC/NUTS

 We call $f_{t}(\theta)$  the density field of Dark Matter at time $t$ which can be paramterised by $\theta$ at initial time. 

 We have a forward model that maps, $f_{0}(\theta)$ with $f_{t_f}(\theta)$, by simulating the dynamic of samples of particles from $f_{0}(\theta)$ between $0$ and $t_f$. 

 We then suppose to have only access to the final density field with some gaussian noise : $$\textbf{data} = f_{t_f}(\theta) + \epsilon.$$

 If our output density field is discretized by some $N$ points $y_i$, this gives for our noised observed data : 
 
$$z_i = y_i(\theta) + \epsilon_i,\quad \epsilon_i \sim \mathcal{N}(0, \sigma^2)$$

Then the likelihood for our observations is :

$$p(\mathbf{z} \mid \theta) = \prod_{i=1}^N \frac{1}{\sqrt{2\pi \sigma^2}} 
\exp\left( - \frac{1}{2\sigma^2} (z_i - y_i(\theta))^2 \right)$$

So finally our log-likelihood for our data is : 
$$\log \mathcal{L}(\theta) = -\frac{1}{2\sigma^2} 
\sum_{i=1}^N (y_i - f_i(\theta))^2 - 
\frac{N}{2}\log(2\pi \sigma^2)$$
